<!-- # CMPUT 200 Fall 2024  Ethics of Data Science and AI
 -->
# Assignment 1: Applying and Analyzing Privacy Techniques

***
- **Dataset**: adult.csv
- **FIRST name**: Eshaan
- **LAST name**: Krishna
- **Student ID**: 1823454

Leave blank if individual:
- **Collaborator names**:
- **Collaborator student IDs**:

**References:**

Kamath, G.[A course on Differential Privacy].(2020 Sept 22).Lecture 3A: Randomized Response[Vedio].YouTube. https://www.youtube.com/watch?v=gCjQ-2W9yjs&list=PLmd_zeMNzSvRRNpoEWkVo6QY_6rR3SHjp&index=6

Kamath, G.[A course on Differential Privacy].(2020 Sept 22).The Definition of Differential Privacy, and Revisiting Randomized Response [Vedio].YouTube. https://www.youtube.com/watch?v=W1v_JfL6sZo&list=PLmd_zeMNzSvRRNpoEWkVo6QY_6rR3SHjp&index=7

Kamath, G.[A course on Differential Privacy].(2020 Sept 23).Lecture 4A: Intro to Differential Privacy, Part 2 - The Laplace Mechanism [Vedio].YouTube. https://www.youtube.com/watch?v=IGQe9BN1TZs&list=PLmd_zeMNzSvRRNpoEWkVo6QY_6rR3SHjp&index=8

geeksforGeeks/gettting started with Jupyter Notebook|Python 2024 Jul 11, accessed 2024 Oct 9, <https://www.geeksforgeeks.org/getting-started-with-jupyter-notebook-python/>.

How to Use Jupyter Notebook: A Beginner’s Tutorial 7 May 2024, accessed 9 Oct 2024, <https://www.dataquest.io/blog/jupyter-notebook-tutorial/>.
***


In this assignment you will apply basic privacy techniques to a dataset of your choosing. By the end of this assignment, you should be able to:
1. Understand and implement randomized response on binary data;
2. Calculate the sensititivity of a non-binary feature;
3. Add noise to a non-binary feature;
4. Compute aggregate statistics.

### Instructions
<!-- **Deadline.**  This assignment is due at ****.  Please check the syllabus for late submissions. -->
You are expected to write clear, detailed, and complete answers when analysing your data. Lack of this may result in point deductions.

**Reminder.** You must submit your own work. The collaboration policy for the assignments is Group Collaboration. You may work together in groups of up to 2.
 On Canvas we added a group set called "Assignment Group".  We have enabled self-signup for you to signup as a group.  If you don't select your group by Wednesday Sep 25th 11:59pm, we will assume you are working on your own.
 Under the group collaboration policy, besides working with your group member, you can discuss concepts with. your peers. However, the work must be by your own group, and all sources of information used including books, websites, students you talked to, must be cited in the submission. Please see the course FAQ document for details on this collaboration policy. We will adhere to current Faculty of Science guidelines on dealing with suspected cases of violation of academic integrity.

You must use this notebook to complete your assignment. You will execute the questions in the notebook. The questions might ask for a short answer in text form or for you to write and execute a piece of code. **Make sure you enter your answer in either case only in the cell provided.**  Do not use a different cell and do not create a new cell. Do not delete any test cells.  Creating new cells for your code and deleting test cells is not compatible with the auto-grading system we are using and thus your assignment will not be graded properly and you will lose marks for that question.

Your submitted notebook should run on our local installation.  So if you are importing packages not listed in the notebook or using local data files not included in the assignment package, make sure the notebook is self-contained with a requirements.txt file or cells in the notebook itself to install the extra packages.  If we cannot run your notebook, you will lose 50% of the marks, and any additional marks that may be lost due to wrong answers.

### Submission Instructions
When you are done, you will submit your work from the notebook. Make sure to save your notebook before running it, and then submit on Canvas the notebook file with your work completed.

**IMPORTANT: Name your file with your *Student ID number* and the assignment number (ex: 1234567_A1.ipynb). Failure to do so will result in a zero!**

In [28]:
# Run this cell to set up; Please don't change this cell.

import numpy as np
from numpy.random import default_rng
rng = default_rng()
import pandas as pd
from scipy.optimize import minimize

# These lines do some fancy plotting magic.
import matplotlib
# This is a magic function that renders the figure in the notebook, instead of displaying a dump of the figure object.
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import warnings
warnings.simplefilter('ignore', FutureWarning)

## Part 1: Data

**Question 1.1.** We will first load the data, carry out some cleaning and pre-processing, and inspect the data to understand what exploratory steps we will take. Name the DataFrame `df`.

In [29]:
# YOUR CODE HERE
# Load the dataset into a DataFrame named df
df = pd.read_csv('adult.csv')

# Replace '?' with NaN for proper handling of missing values
df.replace('?', np.nan, inplace=True)

# Display all column names
print("\nColumn Names:")
print(df.columns.tolist())

# Describe the dataset
description = df.describe(include='all')

print(df.info())

# Display the description
print("\nDataset Description:")
print(description)

# Count the number of categorical, integer, and binary variables
num_categorical = df.select_dtypes(include=['object']).shape[1]
num_integer = df.select_dtypes(include=['int']).shape[1]
num_binary = df.select_dtypes(include=['int', 'object']).apply(lambda x: x.nunique() == 2).sum()

print("\nSummary of the Dataset:")
print(f"Number of Categorical Variables(out of which 2 are Binary): {num_categorical}")
print(f"Number of Integer Variables: {num_integer}")
print(f"Number of Binary Variables: {num_binary}")

# Count and display the number of missing entries for each column
missing_values = df.isnull().sum()

print("\nNumber of Missing Entries for Each Column:")
print(missing_values[missing_values > 0])

# Display the first five rows of the DataFrame to inspect the data
# Describe the shape of dataset
print("Shape: ", df.shape)
df.head(5)


Column Names:
['age', 'workclass', 'fnlwgt', 'education', 'education.num', 'marital.status', 'occupation', 'relationship', 'race', 'sex', 'capital.gain', 'capital.loss', 'hours.per.week', 'native.country', 'income']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             32561 non-null  int64 
 1   workclass       30725 non-null  object
 2   fnlwgt          32561 non-null  int64 
 3   education       32561 non-null  object
 4   education.num   32561 non-null  int64 
 5   marital.status  32561 non-null  object
 6   occupation      30718 non-null  object
 7   relationship    32561 non-null  object
 8   race            32561 non-null  object
 9   sex             32561 non-null  object
 10  capital.gain    32561 non-null  int64 
 11  capital.loss    32561 non-null  int64 
 12  hours.per.week  32561 non-null  int64 
 13  native.co

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
0,90,NaN,77053,HS-grad,9,Widowed,NaN,Not-in-family,White,Female,0,4356,40,United-States,<=50K
1,82,Private,132870,HS-grad,9,Widowed,Exec-managerial,Not-in-family,White,Female,0,4356,18,United-States,<=50K
2,66,NaN,186061,Some-college,10,Widowed,NaN,Unmarried,Black,Female,0,4356,40,United-States,<=50K
3,54,Private,140359,7th-8th,4,Divorced,Machine-op-inspct,Unmarried,White,Female,0,3900,40,United-States,<=50K
4,41,Private,264663,Some-college,10,Separated,Prof-specialty,Own-child,White,Female,0,3900,40,United-States,<=50K


**Question 1.2.** Describe your data and its purpose. Identify one variable that is binary or that could be classified into a binary feature. Identify another that is not a binary feature.

**Data Description and Purpose:**

The dataset used in this project is derived from the U.S. Census and contains information about various individuals, with the purpose of predicting whether an individual's income exceeds $50,000 annually. The dataset is commonly used for training machine learning models in classification tasks, particularly binary classification.

**Overview of the Dataset:**

Number of Entries (Records): 32,561
Number of Features: 15, including both categorical and numerical variables. There are 2 Binary variables.

*Target Variable:* income, which indicates whether an individual's income is greater than $50,000 (">50K") or not ("<=50K").

**Variable Identification:**

*Binary Variable:*
income: This variable is inherently binary as it categorizes individuals into two groups based on their income level: those who earn more than 50K and more and those who earn less than or equal to 50K.

*Non-Binary Variable:*
age: This is a continuous numerical variable that represents the age of individuals in the dataset. It does not fit into a binary classification as it can take on a wide range of values (e.g., from 17 to 90 years).

**Output Data:**

*Column Names*:
The dataset contains the following columns:
age,
workclass,  
fnlwgt,
education,
education.num,
marital.status,
occupation,
relationship,
race,
sex,
capital.gain,
capital.loss,
hours.per.week,
native.country,
income.

**Dataset Description:**

a) The count for each feature indicates the number of non-missing entries.

b) The unique values show the number of distinct values for categorical columns.

c) For numerical columns like age, fnlwgt, education.num, and the capital gain/loss columns, we see the mean, standard deviation, minimum, and maximum values.

**Summary of the Dataset:**

a) *Categorical Variables* (Including 2 Binary): 9 (workclass,education,marital-status, occupation, relationship, race, native-country, sex (bianry type), income (binary type)

b) *Integer Variables:* 6 (includes age, fnlwgt, education.num, capital.gain, capital.loss, hours.per.week)

c) *Binary Variables:* 2 (sex and income)

**Missing Entries:**
A total of three columns have missing entires:

a) workclass: 1836 missing entries

b) occupation: 1843 missing entries

c) native.country: 583 missing entries

These missing entries might need to be handled.

YOUR ANSWER HERE

## Part 2: Data pre-processing

**Question 2.1.** If your data has missing values or empty rows, remove them in the cell below. If the feature that you chose above has to be classified into a binary feature, convert it. Finally, if your binary feature is categorical, convert it to numerial.

In [30]:
# YOUR CODE HERE
# Drop rows with missing values
df.dropna(inplace=True)

# Convert the binary categorical variable 'income' to numerical format
df['income'] = df['income'].map({'<=50K': 0, '>50K': 1})

# Convert the binary categorical variable 'sex' to numerical format
df['sex'] = df['sex'].map({'Female': 0, 'Male': 1})

print("Shape: ", df.shape)

# Display the first few rows to verify changes
df.head()

Shape:  (30162, 15)


,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
1,82,Private,132870,HS-grad,9,Widowed,Exec-managerial,Not-in-family,White,0,0,4356,18,United-States,0
3,54,Private,140359,7th-8th,4,Divorced,Machine-op-inspct,Unmarried,White,0,0,3900,40,United-States,0
4,41,Private,264663,Some-college,10,Separated,Prof-specialty,Own-child,White,0,0,3900,40,United-States,0
5,34,Private,216864,HS-grad,9,Divorced,Other-service,Unmarried,White,0,0,3770,45,United-States,0
6,38,Private,150601,10th,6,Separated,Adm-clerical,Unmarried,White,1,0,3770,40,United-States,0


## Part 3: Randomized Response

Now let's implement a Randomized Response mechanism. First we have to identify what the query on our **binary** variable will be. Then we can create our own randomized mechanism.

**Question 3.1.** Write a query on your binary feature.

"What proportion of individuals in the dataset have an income greater than 50K?"

YOUR ANSWER HERE

**Question 3.2.** Create your own randomized response mechanism for the query you defined above. You may NOT use the coin example from class, try to be creative!

***Randomized Response Mechanism: ***

Color Card Method:

We can use a color card method where respondents have three cards: a green card, a red card, and a blue card. Each color card represents a different type of response:

a) Green Card: The respondent answers truthfully regarding their income status.

b) Red Card: The respondent gives a predetermined random answer (e.g., they always respond "more than 50K").

c) Blue Card: The respondent gives a different predetermined random answer (e.g., they always respond "less than or equal to 50K").

**Implementation Steps:**

Determine Card Distribution: Decide the probability for each card.

50% chance of receiving a green card (truthful response)

25% chance of receiving a red card (randomly stating "more than 50K")

25% chance of receiving a blue card (randomly stating "less than or equal to 50K")

**Generate Random Responses:**

 For each respondent, randomly select a card based on the defined probabilities and determine the response accordingly.

**Question 3.3.** Implement a function for your mechanism in 3.2. in the code cell below. The function should accept a value `0` or `1` and return the reported answer according to the randomized response above. Name your function `rand_resp`.

In [31]:
# YOUR CODE HERE
import random

def rand_resp(income):
    """
    Parameters: income (int): The income value (0 for <=50K, 1 for >50K).
    Returns: int: The reported income status based on the randomized response.
    """
    # Define probabilities and responses
    responses = ['truthful', 'greater_than_50K', 'less_than_equal_to_50K']
    # 50% truthful, 25% greater than 50K, 25% less than or equal to 50K
    probabilities = [0.5, 0.25, 0.25]

    # Randomly select a card based on the defined probabilities
    card = random.choices(responses, probabilities)[0]

    # Determine the response based on the card selected
    if card == 'truthful':
        return income               # Respond truthfully
    elif card == 'greater_than_50K':
        return 1                    # Respond "more than 50K"
    else:  # card == 'less_than_equal_to_50K'
        return 0                    # Respond "less than or equal to 50K"


**Question 3.4.** For each value in your dataframe's binary feature column, call your function. Store the results in a new column in df named `rrc1`.

In [39]:
# YOUR CODE HERE

# Apply the rand_resp function to the 'income' column and store the results in a new column 'rrc1'
df['rrc1'] = df['income'].apply(rand_resp)

# Display the updated DataFrame to verify the new column
df[['income', 'rrc1']].head()

,income,rrc1
1,0,1
3,0,0
4,0,0
5,0,0
6,0,1


**Question 3.5.** Now get the **estimate** for the true number of people who answered `1`.  Write this result into the variable `count_est_true_yes`.  Given the number of reported `1`'s, we know how to estimate the proportion or number of true `1`'s.

Calculate the true number of people who answered `1` (the true answer in the data we imported) and write it into the variable `count_true_yes`.

In [40]:
# YOUR CODE HERE
# Count reported 1's in the randomized response column (rrc1)
reported_yes_count = df['rrc1'].sum()

# Assuming the probability of randomizing responses was p = 0.5
p = 0.5

# Estimate the true count of 1's based on the number of reported 1's
count_est_true_yes = reported_yes_count / (1 - p)

# Count true 1's in the original income column
count_true_yes = df['income'].sum()  # Assuming income = 1 for '>50K'

# Print the results

print("Estimated true yes count: ", count_est_true_yes)
print("True yes count: ", count_true_yes)

Estimated true yes count:  22566.0
True yes count:  7508


**Question 3.6.** Comment on your results from above. What can you say about the privacy-accuracy tradeoff of your randomized response mechanism?

The results obtained from the randomized response mechanism show a significant disparity between the estimated true count of individuals earning more than $50K (22566) and the actual count (7,508). This discrepancy raises several points for consideration regarding the effectiveness of the randomized response approach:

**Overestimation:**


The estimated count of individuals earning over $50K is approximately three times higher than the actual count. This substantial overestimation indicates that the randomness incorporated into the response mechanism may have led to misleading results. Such a large gap suggests that the mechanism did not effectively capture the true distribution of responses.

**Implications of Randomness:**


The randomized response method introduces randomness to protect the privacy of respondents. However, this randomness can dilute the accuracy of the estimates. If the probabilities used in the mechanism do not reflect the actual behavior of respondents, the resulting data may not be reliable.
The mechanism may inadvertently inflate the number of affirmative responses, thereby skewing the estimates upward.

**Privacy-Accuracy Tradeoff**

The randomized response mechanism illustrates a fundamental tradeoff between privacy and accuracy in survey data collection:

**Enhanced Privacy:**

One of the key benefits of using randomized response is that it allows respondents to provide answers to sensitive questions without fear of exposure. This can encourage more honest responses and increase participation in surveys concerning sensitive topics such as income.
In contexts where confidentiality is paramount, such mechanisms can be invaluable for obtaining data while protecting individuals' identities.

**Reduced Accuracy:**

While privacy is enhanced, the accuracy of the results may suffer. The randomization may result in a significant deviation from the actual values, as demonstrated by the estimated count compared to the true count.

In this case, the substantial overestimate indicates that the randomized response mechanism could mislead policymaker, potentially leading to erroneous conclusions or decisions based on inaccurate data.

**Finding the Balance:**

Striking the right balance between privacy and accuracy is crucial for effective data collection. Adjustments to the randomization probabilities and further refinements to the methodology could help improve accuracy while maintaining a level of respondent confidentiality.

While the randomized response mechanism enhances privacy for sensitive questions, the results from your analysis highlight a significant tradeoff with accuracy. The substantial overestimation of the true count suggests that further refinement of the mechanism is necessary to obtain reliable estimates. Future implementations should aim to balance these competing priorities, ensuring that data collected through such methods remains both actionable and respectful of respondents' privacy.

YOUR ANSWER HERE

**Question 3.7.** We learned in class that data analysts are still able to obtain aggregate statistics from the results of a randomized response survey. Using the column you made above, `rrc1`, to calculate the mean and median for the estimated true responses. Do the same for your true responses. Name your answers `mean_est_true_yes`, `mean_true_yes`, `median_est_true_yes`, and `median_true_yes` respectively.

In [41]:
# YOUR CODE HERE

# Calculate mean and median for the estimated true responses (from rrc1)
mean_est_true_yes = df['rrc1'].mean()
median_est_true_yes = df['rrc1'].median()

# Calculate mean and median for the true responses (from income)
mean_true_yes = df['income'].mean()
median_true_yes = df['income'].median()

# Display the results
print("Mean: ", mean_est_true_yes, mean_true_yes)
print("Median: ", median_est_true_yes, median_true_yes)

Mean:  0.3740799681718719 0.24892248524633645
Median:  0.0 0.0


**Question 3.8.** Comment on your results from above. Are the results from your mechanism useable? What can you say about the privacy when it comes to randomized response? Comment on the distributions of your data.

**Mean Values:**

Estimated Mean (from rrc1): Approximately 0.373 indicates that around 37.1% of the randomized responses indicated a "yes" (i.e., answered 1).
True Mean (from income): Approximately 0.249 indicates that around 24.9% of the actual responses were "yes" (i.e., income >50K).

The estimated mean is significantly higher than the true mean, which could suggest that the randomized response mechanism may have introduced a bias towards higher "yes" responses.

**Median Values:**
Estimated Median (from rrc1): 0.0, indicating that at least half of the randomized responses were "no" (i.e., answered 0).
True Median (from income): Also 0.0, which suggests that half of the actual responses are "no" (i.e., income ≤50K).

The median results align between both the estimated and true responses, indicating that the middle point in the responses is similar, but it also reflects that a substantial number of responses, both estimated and actual, lean towards "no."

**Usefulness of the Results:**
The mean estimate from the randomized response mechanism is not directly usable for precise conclusions because it overestimates the number of "yes" responses. However, the median is more robust and offers some reliability, indicating that the mechanism might still provide some value in understanding general trends.

**Privacy Implications**

**Randomized Response Mechanism: **

This method effectively maintains privacy by allowing individuals to provide responses without revealing their actual answers. The uncertainty introduced helps protect individual privacy, but it also complicates the accuracy of the results.

In this case, while the true responses remain confidential, the aggregation of data through means or medians may still give analysts a sense of overall trends while protecting individual data.

**Distribution of Data:**
*The results indicate a skewed distribution:*
The mean being higher than the median suggests a right skew, where more individuals fall under the "no" category, but a notable proportion of "yes" responses are impacting the average.

Given that both the estimated and actual median values are 0.0, it shows a concentration of individuals who fall into the "no" category, which could indicate a prevalent characteristic within the dataset (e.g., most individuals earn less than or equal to $50K).

While the randomized response mechanism serves well for privacy, analysts must be cautious with the results, especially means, as they may not reflect true values accurately. The median remains a viable measure for understanding central tendencies within the dataset.

YOUR ANSWER HERE

## Part 4: Adding Noise

We are going to use the non-binary feature that you chose earlier and add noise to it. To do this, we apply the same steps that we would for differential privacy: $f(D) + Z$.

**Question 4.1.** Suppose the function we wish to query is **count**. What would the global sensitivity, $S(f)$, be? Explain why.

**For the Count Function:**
Definition: The count function simply returns the number of entries in the dataset.

**Changing the Dataset:**

If you have a dataset D with n entries:

a) Adding one entry results in f(D)=n and f(D')=n+1.

b) Removing one entry results in f(D)=n and f(D')=n-1.

**Calculating the Difference:**

a) The change when you add one entry is: ∣f(D)−f(D′)∣=∣n−(n+1)∣=1

b) The change when you remove one entry is:∣f(D)−f(D')∣=∣n−(n−1)∣=1

**Conclusion:**
No matter whether you add or remove one entry, the count can only change by 1.

So, the global sensitivity
S(f) for the count function is: S(f)=1

YOUR ANSWER HERE

**Question 4.2.** In the cell below, write a function that adds Laplace noise to a value given the sensitivity and the privacy parameter, epsilon. Name your function `add_laplace_noise`.

In [42]:
# YOUR CODE HERE
def add_laplace_noise(value, sensitivity, epsilon):
    """
   Parameters:
    - value: The original value to which noise will be added.
    - sensitivity: The global sensitivity of the function.
    - epsilon: The privacy parameter, controlling the amount of noise.
    Returns:
    - The value with added Laplace noise.
    """
    # Calculate the scale of the Laplace distribution
    scale = sensitivity / epsilon

    # Generate Laplace noise
    laplace_noise = np.random.laplace(0, scale)

    # Add noise to the original value
    noisy_value = value + laplace_noise

    return noisy_value


Since our query is on count, we'll have to obtain the count of each unique value of our feature.

**Question 4.3.** Define a variable called that holds the count of each unique value of your feature. *Hint: there's a method that does this for us!*

In [43]:
# YOUR CODE HERE
age_counts = df['age'].value_counts()

# Display the counts
print(age_counts)

age
36    852
31    851
33    837
34    836
37    828
     ... 
82      7
83      5
88      3
85      3
86      1
Name: count, Length: 72, dtype: int64


**Question 4.4.** Before we add noise, let's calculate some stats from your variable from 4.3. Calculate the mean, median, and count and name them `mean_count`, `median_count`, and `total_count` respectively.

In [44]:
# YOUR CODE HERE
# Calculate mean, median, and total count from education_counts
mean_count = age_counts.mean()
median_count = age_counts.median()
total_count = age_counts.sum()

print(f"Mean of true counts: {mean_count}")
print(f"Median of true counts: {median_count}")
print(f"Total count of true records: {total_count}")

Mean of true counts: 418.9166666666667
Median of true counts: 420.5
Total count of true records: 30162


**Question 4.5.** Now we can start adding noise. Use your Laplace function and your variable from 4.3. to calculate a noisy representation of each value in your feature. Set your value of epsilon to 1 for now.

In [45]:
# YOUR CODE HERE
# Define global sensitivity for the count function
S_f = 1  # Sensitivity is 1 for count

# Set the privacy parameter epsilon
epsilon = 1

# Initialize an empty dictionary to hold noisy counts
noisy_age_counts = {}

# Iterate over each unique value and its count in education_counts
for value, count in age_counts.items():
    # Add Laplace noise to each count
    noisy_count = add_laplace_noise(count, S_f, epsilon)  # Using S_f as the sensitivity
    noisy_age_counts[value] = noisy_count

# Display the noisy counts
print("Noisy Age Counts:", noisy_age_counts)

Noisy Age Counts: {36: 852.1224831834274, 31: 850.3370039010914, 33: 835.7961015924325, 34: 836.3663494149832, 37: 827.7367218748868, 35: 829.1139634942735, 23: 824.1037179166807, 30: 812.3758890661038, 28: 807.4098681331936, 25: 799.1256619632351, 38: 790.1502093897918, 27: 787.8252061539592, 32: 789.3574170889495, 39: 786.8996460059242, 29: 773.9307994809465, 41: 769.6435156323867, 40: 764.0808851421396, 24: 754.8279535357467, 26: 743.4563182215326, 43: 743.8904835140869, 42: 739.9702439388627, 46: 711.8773453766446, 45: 704.594402243458, 44: 704.1939998328189, 47: 683.4232937113045, 22: 673.2837568004076, 20: 628.9455852612169, 21: 623.2249259804222, 19: 595.6935806176086, 50: 574.1363296200788, 51: 570.8960435216204, 49: 555.9064391484011, 48: 524.7057880326031, 52: 454.111929799031, 53: 448.4278712626862, 18: 446.90197881390554, 54: 395.41389156281565, 55: 385.48523972381577, 58: 342.84151103249934, 56: 343.04320987662675, 57: 338.6452272897918, 59: 331.6999134875753, 17: 329.5419

**Question 4.6.** Now calculate the stats for your noisy values. Calculate the mean, median, and count and name them `mean_noisy_count`, `median_noisy_count`, and `total_noisy_count` respectively.

In [46]:
# YOUR CODE HERE
# Get the noisy counts as a list
noisy_counts_list = list(noisy_age_counts.values())

# Calculate mean, median, and total count for noisy counts
mean_noisy_count = np.mean(noisy_counts_list)
median_noisy_count = np.median(noisy_counts_list)
total_noisy_count = len(noisy_counts_list)  # Total count of unique age levels

print(f"Noisy mean of true counts: {mean_noisy_count}")
print(f"Noisy median of true counts: {median_noisy_count}")
print(f"Total noisy count of true records: {total_noisy_count}")

Noisy mean of true counts: 418.9551915110403
Noisy median of true counts: 421.1579351883606
Total noisy count of true records: 72


**Question 4.7.** Comment on the differences in aggregate statistics between the original and the noisy values. What can you say about the utility and privacy?  

Analysis of Differences
**Mean Comparison:**

The true mean (418.92) and the noisy mean (419.07) are quite close, showing that the added noise had a minimal effect on the mean value. This indicates that the mechanism successfully preserves the overall average of the dataset while providing privacy.
The small difference suggests that the noise added is relatively low compared to the actual values, maintaining a good level of utility for aggregate statistical measures.

**Median Comparison:**

The true median (420.5) and the noisy median (420.58) are also very similar. This close proximity indicates that the central tendency of the data remains intact despite the introduction of noise.
The median being less sensitive to extreme values means that the impact of noise on this statistic is minimal, which is a favorable outcome for ensuring utility.

**Count Comparison:**

The total count of true records (30,162) and the total noisy count of true records (72) indicate a significant reduction in the number of records used for the noisy estimates. This drastic reduction could imply that the process for selecting or filtering records (or a threshold for reporting) might be too stringent or that the noise addition has led to a loss of many data points, which could affect the robustness of the noisy statistics.
The substantial difference in counts raises concerns about the overall utility of the dataset; a smaller sample size can lead to less reliable conclusions.

**Utility vs. Privacy:**

**Utility:**

The close alignment of means and medians suggests that the differential privacy mechanism has managed to maintain useful aggregate statistics, providing reliable information about the dataset without disclosing individual data points.
However, the drastic drop in the total count of records raises concerns about the overall utility of the noisy dataset. Fewer records mean that the noisy estimates may not fully represent the underlying population, leading to potentially misleading interpretations.

**Privacy:**

The success in preserving the overall distribution while adding noise indicates effective privacy protection. The mechanism seems to be operating well to obscure individual data points, which is critical in maintaining respondent confidentiality.
The added noise is sufficient to protect privacy without drastically altering the statistical properties of the dataset, which is a key objective in differential privacy.

While the mechanism has performed well in maintaining the overall statistical properties of the data (mean and median), the significant reduction in the total count of records used for noisy estimates raises concerns about the utility of the noisy dataset. This indicates a potential tradeoff: while privacy is adequately protected, the robustness and reliability of the resulting statistics may be compromised due to the loss of data points.

**Question 4.8.** Go back to question 4.5. and change the value of epsilon. Repeat this until you notice a pattern in your aggregate statistics. What happens as epsilon changes? What happens to the privacy and the accuracy?

**Observing the Changes:**

Effect of Epsilon on Noisy Counts:

Low :
Epsilon (e.g., 0.1): The noisy counts will be more dispersed and may deviate significantly from the true counts due to the larger noise. You may see less reliability in these counts.

Medium:
Epsilon(e.g., 1): The noisy counts should start to stabilize and become more accurate as the noise is less extreme.

High:
Epsilon (e.g., 5): The noisy counts will likely be very close to the true counts because less noise is added, leading to high accuracy.

**Privacy Implications:**
As Epsilon decreases, the privacy protection increases. The added noise obscures individual data points more effectively, reducing the risk of re-identification.

Conversely, as Epsilon increases, privacy decreases. With less noise, there's a greater chance of inferring individual responses from the noisy counts.

**Accuracy Tradeoff:**

Lower Epsilon values lead to noisy counts that might not accurately represent the true data, thereby affecting utility. However, they provide stronger privacy.

Higher Epsilon values yield more reliable statistics at the cost of increased risk of exposing individual data points.

Through this systematic examination of Epsilon, we gain insights into the relationship between privacy and accuracy in your differential privacy mechanism. This will enable you to make informed decisions about the appropriate level of privacy for your analysis, balancing the need for reliable data with the imperative to protect individual privacy.

# Rubric

| Question | Points|
|----------|----------|
| 1.1.   | 5   |
| 1.2.    | 10   |
| 2.1.    | 5   |
| 3.1.   | 5   |
| 3.2.    | 10  |
| 3.3.  | 10   |
| 3.4.    | 2   |
| 3.5.   | 8   |
| 3.6.   | 5   |
| 3.7.   | 6   |
| 3.8.   | 8   |
| 4.1.   | 2   |
| 4.2.    | 10  |
| 4.3.  | 5   |
| 4.4.    | 3   |
| 4.5.   | 5   |
| 4.6.   | 3   |
| 4.7.   | 5   |
| 4.8.   | 8   |
| Total  | 115   |


